<a href="https://colab.research.google.com/github/kesakeerthi/hello-world/blob/master/PySpark_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark General Usages

## Download and install Spark

In [4]:
!ls


drive  sample_data


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


## Setup environment

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

## Downloading and preprocessing Chicago's Reported Crime Data

In [8]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
!ls -l

--2020-02-02 03:01:24--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.199, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [  <=>               ]   1.55G  3.04MB/s    in 8m 44s  

2020-02-02 03:10:09 (3.03 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1664670357]

total 1850372
drwx------  4 root root       4096 Feb  2 03:00  drive
-rw-r--r--  1 root root 1664670357 Feb  1 12:13 'rows.csv?accessType=DOWNLOAD'
drwxr-xr-x  1 root root       4096 Jan 13 16:38  sample_data
drwxr-xr-x 13 1000 1000       4096 Aug 27 21:30  spark-2.4.4-bin-hadoop2.7
-rw-r--r--  1 root root  230091034 Aug 27 22:01  spark-2.4.4-bin-hadoop2.7.tgz


In [9]:
!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv
!ls -l

total 1850372
drwx------  4 root root       4096 Feb  2 03:00 drive
-rw-r--r--  1 root root 1664670357 Feb  1 12:13 reported-crimes.csv
drwxr-xr-x  1 root root       4096 Jan 13 16:38 sample_data
drwxr-xr-x 13 1000 1000       4096 Aug 27 21:30 spark-2.4.4-bin-hadoop2.7
-rw-r--r--  1 root root  230091034 Aug 27 22:01 spark-2.4.4-bin-hadoop2.7.tgz


In [23]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') < lit('2018-11-11'))
rc.show(5)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11| 

## Schemas

In [11]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [12]:
rc.select('IUCR').show(5)

+----+
|IUCR|
+----+
|1153|
|0281|
|0620|
|0810|
|0281|
+----+
only showing top 5 rows



In [13]:
rc.select(col('IUCR')).show(5)

+----+
|IUCR|
+----+
|1153|
|0281|
|0620|
|0810|
|0281|
+----+
only showing top 5 rows



In [14]:
rc.select(rc.IUCR).show(5)

+----+
|IUCR|
+----+
|1153|
|0281|
|0620|
|0810|
|0281|
+----+
only showing top 5 rows



In [15]:
rc.select('Case Number', 'Date', 'Arrest').show(4)

+-----------+-------------------+------+
|Case Number|               Date|Arrest|
+-----------+-------------------+------+
|   JA366925|2001-01-01 11:00:00| false|
|   JB147188|2017-10-08 03:00:00| false|
|   JB147595|2017-03-28 14:00:00| false|
|   JB147230|2017-09-09 20:17:00| false|
+-----------+-------------------+------+
only showing top 4 rows



### Add new column name Ones with all entries 1 -- Literals

In [0]:
from pyspark.sql.functions import lit

In [17]:
rc.withColumn('Ones', lit(1)).show(5)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+----+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|Ones|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+----+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|          

In [18]:
rc = rc.drop('IUCR')
rc.show(5)

+--------+-----------+-------------------+--------------------+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|        null|        

### Working with Rows

In [0]:
one_day = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') == lit('2018-11-11'))

In [20]:
one_day.count()

11

### Add the reported crimes for an additional day, 11-Nov-2018, to our dataset.

In [24]:
rc.union(one_day).orderBy('Date', ascending=False).show(15)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11513303|   JB523990|2018-11-11 00:00:00|  007XX S CICERO AVE|0281|CRIM SEXUAL ASSAULT|      NON-AGGRAVATED|              STREET|

### What are the top 10 number of reported crimes by Primary type, in descending order of occurence?

In [25]:
rc.groupby('Primary Type').count().orderBy('count', ascending = False).show(10)

+-------------------+-------+
|       Primary Type|  count|
+-------------------+-------+
|              THEFT|1418410|
|            BATTERY|1232212|
|    CRIMINAL DAMAGE| 771496|
|          NARCOTICS| 711604|
|      OTHER OFFENSE| 418817|
|            ASSAULT| 418502|
|           BURGLARY| 388028|
|MOTOR VEHICLE THEFT| 314143|
| DECEPTIVE PRACTICE| 265252|
|            ROBBERY| 255600|
+-------------------+-------+
only showing top 10 rows



### Challenge - 1: What percentage of reported crimes resulted in an arrest?

In [26]:
rc.select(rc.Arrest).distinct().show()

+------+
|Arrest|
+------+
| false|
|  true|
+------+



In [27]:
rc.groupby('Arrest').count().show()

+------+-------+
|Arrest|  count|
+------+-------+
| false|4878236|
|  true|1873964|
+------+-------+



In [0]:
rc.filter(rc.Arrest == 'true').count() * 100 / rc.count()

### Challenge - 2: What are the top 3 locations for reported crimes?

In [0]:
rc.groupby('Location Description').count().orderBy('count', ascending = False).show(3)

### Built-in Functions

In [0]:
from pyspark.sql import functions
print(dir(functions))

### String Functions

Display Primary type - Lower, Upper, First 4 Chars

In [0]:
rc.select('Primary Type').show(10)

In [0]:
from pyspark.sql.functions import lower, upper, substring
rc.select(lower(col('Primary Type'))).show(10)

In [0]:
rc.select(substring(col('Primary Type'),1,4)).show(10)

## Numeric Functions

Show Oldest and Recent date

In [0]:
from pyspark.sql.functions import min, max

rc.select(min(col('Date')), max(col('Date'))).show()

3 days earlier the  Oldest date and 1 month later the Recent date

In [0]:
from pyspark.sql.functions import date_add, date_sub
rc.select(  date_sub(min(col('Date')), 3), date_add(max(col('Date')),30 )         ).show()

## Working with Dates - Detail



In [0]:
from pyspark.sql.functions import to_date, to_timestamp, lit

In [0]:
# 1984-10-25 21:11:26
df = spark.createDataFrame( [ ('1984-10-25 21:11:26', ) ] , ['Birthday']       )
df.show()

In [0]:
df.select(to_date(col('Birthday') , 'yyyy-MM-dd HH:mm:ss'), to_timestamp(col('Birthday') , 'yyyy-MM-dd HH:mm:ss')).show(1)

In [0]:
# 25/Oct/1984 21:11:26
df = spark.createDataFrame( [ ('25/Oct/1984 21:11:26', ) ] , ['Birthday']       )
df.select(to_date(col('Birthday') , 'dd/MMM/yyyy HH:mm:ss'), to_timestamp(col('Birthday') , 'dd/MMM/yyyy HH:mm:ss')).show(1)



In [0]:
# 10/25/1984 09:11:26 PM
df = spark.createDataFrame( [ ('10/25/1984 09:11:26 PM', ) ] , ['Birthday']       )
df.select(to_date(col('Birthday') , 'MM/dd/yyyy hh:mm:ss aa'), to_timestamp(col('Birthday') , 'MM/dd/yyyy hh:mm:ss aa')).show(1)

## JOINS

In [35]:
!wget -O police-station.csv https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD

--2020-02-02 03:22:07--  https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.68.26, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘police-station.csv’

police-station.csv      [ <=>                ]   5.57K  --.-KB/s    in 0s      

2020-02-02 03:22:07 (529 MB/s) - ‘police-station.csv’ saved [5699]



In [34]:
!ls


drive		     sample_data		spark-2.4.4-bin-hadoop2.7.tgz
reported-crimes.csv  spark-2.4.4-bin-hadoop2.7


In [0]:
ps = spark.read.csv('police-station.csv', header = True)
ps.show(5)

In [0]:
rc.cache()
rc.count()

In [0]:
ps.select(col('DISTRICT')).distinct().show()

In [0]:
rc.select(col('DISTRICT')).distinct().show()

In [0]:
from pyspark.sql.functions import lpad

In [0]:
help(lpad)

In [0]:
ps.select(lpad(col('DISTRICT'),3,'0')).show(10)

In [0]:
ps = ps.withColumn('Format_District',lpad(col('DISTRICT'),3,'0') )
ps.show(5)

In [0]:
rc.join(ps, rc.District == ps.Format_District, 'left_outer').show(5)

## RDDs

In [36]:
psrdd = sc.textFile('police-station.csv')
psrdd.first()

'DISTRICT,DISTRICT NAME,ADDRESS,CITY,STATE,ZIP,WEBSITE,PHONE,FAX,TTY,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION'

In [0]:
ps_header = psrdd.first()

In [39]:
ps_rest = psrdd.filter(lambda line: line!= ps_header)
ps_rest.first()

'Headquarters,Headquarters,3510 S Michigan Ave,Chicago,IL,60653,http://home.chicagopolice.org,,,,1177731.401,1881697.404,41.83070169,-87.62339535,"(41.8307016873, -87.6233953459)"'

### How many Police stations are there?

In [40]:
ps_rest.map(lambda line: line.split(',')).collect()

[['Headquarters',
  'Headquarters',
  '3510 S Michigan Ave',
  'Chicago',
  'IL',
  '60653',
  'http://home.chicagopolice.org',
  '',
  '',
  '',
  '1177731.401',
  '1881697.404',
  '41.83070169',
  '-87.62339535',
  '"(41.8307016873',
  ' -87.6233953459)"'],
 ['1',
  'Central',
  '1718 S State St',
  'Chicago',
  'IL',
  '60616',
  'http://home.chicagopolice.org/community/districts/1st-district-central/',
  '312-745-4290',
  '312-745-3694',
  '312-745-3693',
  '1176569.052',
  '1891771.704',
  '41.85837259',
  '-87.62735617',
  '"(41.8583725929',
  ' -87.627356171)"'],
 ['6',
  'Gresham',
  '7808 S Halsted St',
  'Chicago',
  'IL',
  '60620',
  'http://home.chicagopolice.org/community/districts/6th-district-gresham/',
  '312-745-3617',
  '312-745-3649',
  '312-745-3639',
  '1172283.013',
  '1853022.646',
  '41.75213684',
  '-87.64422891',
  '"(41.7521368378',
  ' -87.6442289066)"'],
 ['11',
  'Harrison',
  '3151 W Harrison St',
  'Chicago',
  'IL',
  '60612',
  'http://home.chicagopol

In [45]:
ps_rest.map(lambda line: line.split(',')).count()

24

### Display the  District Id,  District Name, Address and Zip for the Police station  with District Id

In [46]:
(ps_rest.filter(lambda line: line.split(',')[0] == '7').
 map(lambda line: (line.split(',')[0],
                   line.split(',')[1],
                   line.split(',')[2],
                   line.split(',')[5],
                   )).collect())

[('7', 'Englewood', '1438 W 63rd St', '60636')]

### Police stations 10 and 11 are geographically close to each other. Display District Id, District Name, Address and Zip for the Police station with District Id

In [47]:
(ps_rest.filter(lambda line: line.split(',')[0] in['10','11'] ).
 map(lambda line: (line.split(',')[0],
                   line.split(',')[1],
                   line.split(',')[2],
                   line.split(',')[5],
                   )).collect())

[('11', 'Harrison', '3151 W Harrison St', '60612'),
 ('10', 'Ogden', '3315 W Ogden Ave', '60623')]